<a href="https://colab.research.google.com/github/sharvaniyadav/Automated-Component-Detection-in-PCB-Manufacturing-Using-YOLOv8-Project-3/blob/main/FinalRunOfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install Required Libraries
!pip install ultralytics --quiet
!pip install opencv-python-headless matplotlib Pillow --quiet

# Import Libraries
from google.colab import drive
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
from IPython.display import Image, display

'''Step 1: Object Masking'''

# Step 1.1: Check and Mount Google Drive to Access the Dataset
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

# Step 1.2: Define the Path to Your Image and Load It
image_path = '/content/drive/MyDrive/data/motherboard_image.JPEG'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)
if image is None:
    raise FileNotFoundError(f"Image at {image_path} could not be loaded. Check the file path.")

# Display the Original Image
plt.figure(figsize=(8, 8))
plt.title("Original Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# Step 1.3: Convert to Grayscale
grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display Grayscale Image
plt.figure(figsize=(8, 8))
plt.title("Grayscale Image")
plt.imshow(grayscale_image, cmap='gray')
plt.axis("off")
plt.show()

# Step 1.4: Enhance Contrast
contrast_enhanced_image = cv2.equalizeHist(grayscale_image)

# Display Enhanced Contrast Image
plt.figure(figsize=(8, 8))
plt.title("Enhanced Contrast")
plt.imshow(contrast_enhanced_image, cmap='gray')
plt.axis("off")
plt.show()

# Step 1.5: Apply Gaussian Blur
blurred_image = cv2.GaussianBlur(contrast_enhanced_image, (5, 5), 0)

# Display Blurred Image
plt.figure(figsize=(8, 8))
plt.title("Blurred Image")
plt.imshow(blurred_image, cmap='gray')
plt.axis("off")
plt.show()

# Step 1.6: Perform Thresholding
_, binary_image = cv2.threshold(
    blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
)

# Step 1.7: Morphological Cleaning
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
cleaned_binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

# Step 1.8: Detect and Filter Contours
contours, _ = cv2.findContours(
    cleaned_binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)
filtered_contours = [c for c in contours if cv2.contourArea(c) > 50000]

if not filtered_contours:
    raise ValueError("No significant contours found. Consider adjusting parameters.")

# Step 1.9: Create a Mask
pcb_mask = np.zeros_like(grayscale_image)
cv2.drawContours(pcb_mask, filtered_contours, -1, 255, cv2.FILLED)

# Step 1.10: Refine Mask Boundaries
refinement_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
refined_mask = cv2.erode(pcb_mask, refinement_kernel, iterations=1)

# Step 1.11: Extract the PCB
extracted_pcb = cv2.bitwise_and(image, image, mask=refined_mask)

# Step 1.12: Crop the PCB
x, y, w, h = cv2.boundingRect(max(filtered_contours, key=cv2.contourArea))
cropped_pcb = extracted_pcb[y:y+h, x:x+w]

# Step 1.13: Visualize Results
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.title("Grayscale Image")
plt.imshow(cv2.cvtColor(grayscale_image, cv2.COLOR_GRAY2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Mask")
plt.imshow(refined_mask, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Extracted PCB")
plt.imshow(cv2.cvtColor(cropped_pcb, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()

'''Step 2: YOLOv8 Training'''

# Step 2.1: Import Necessary Libraries
print("Libraries imported successfully.")

# Step 2.2: Load the YOLOv8 Nano Model
model = YOLO('yolov8n.pt')
print("YOLOv8 Nano model loaded with pretrained weights.")

# Step 2.3: Set Up Training Configuration
dataset_config_path = '/content/drive/MyDrive/data/data.yaml'

# Training parameters
total_epochs = 180
images_per_training_batch = 8
initial_learning_rate_value = 0.0004
type_of_optimizer_used = 'AdamW'
regularization_weight_value = 0.001

# Data-related parameters
resolution_of_input_images = 960
enable_data_augmentation_process = True
enable_mixed_precision_training_mode = True

# Custom output directory in Google Drive
output_directory = '/content/drive/MyDrive/data/pcb_training_results/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created directory: {output_directory}")
else:
    print(f"Directory already exists: {output_directory}")

# Step 2.4: Start Model Training
print("Initiating the training phase. Please allow some time for completion.")

result = model.train(
    name='pcb_model_training_process',
    data=dataset_config_path,
    imgsz=resolution_of_input_images,
    batch=images_per_training_batch,
    epochs=total_epochs,
    lr0=initial_learning_rate_value,
    optimizer=type_of_optimizer_used,
    amp=enable_mixed_precision_training_mode,
    augment=enable_data_augmentation_process,
    weight_decay=regularization_weight_value,
    device=0,
    save_dir=output_directory
)

# Capture the output directory for trained weights
training_output_dir = result['save_dir']
trained_model_path = os.path.join(training_output_dir, 'weights/best.pt')

# Check if the model exists
if not os.path.exists(trained_model_path):
    raise FileNotFoundError(f"Trained model not found at: {trained_model_path}")

print("Model training has been successfully completed!")

'''Step 3: YOLOv8 Evaluation'''

# Paths to evaluation images
image_paths = [
    '/content/drive/MyDrive/data/evaluation/ardmega.jpg',
    '/content/drive/MyDrive/data/evaluation/arduno.jpg',
    '/content/drive/MyDrive/data/evaluation/rasppi.jpg'
]

# Verify evaluation images exist
for img_path in image_paths:
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Evaluation image not found at: {img_path}")

# Define directory to save evaluation results
evaluation_output_dir = os.path.join(training_output_dir, 'evaluation_results/')
os.makedirs(evaluation_output_dir, exist_ok=True)

# Perform predictions for each evaluation image and save the results
for img_path in image_paths:
    print(f"Evaluating: {img_path}")
    model.predict(source=img_path, save=True, save_dir=evaluation_output_dir)

# Confirm saved results directory
print(f"Evaluation results saved in: {evaluation_output_dir}")


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968